<a href="https://colab.research.google.com/github/AIandAutomationTools/ai-quartet-evaluator/blob/main/ai_quartet_cleaned_pitch_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎵 AI Quartet Pitch Comparison
This notebook compares a student's singing to a professor's reference using pitch analysis and visualization.

In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import requests
import io
import os

In [ ]:
def load_audio_from_url(url):
    if not url:
        raise ValueError("❌ Missing URL input.")
    try:
        print(f"🔗 Attempting to download: {url}")
        response = requests.get(url)
        response.raise_for_status()
        audio_data = io.BytesIO(response.content)
        audio, sr = librosa.load(audio_data, sr=None)
        return audio, sr
    except Exception as e:
        raise RuntimeError(f"❌ Failed to load audio from {url}\nError: {e}")

In [ ]:
# Get URLs from environment variables (used by Zapier/Colab)
#professor_url = os.environ.get("professor_url")
#student_url = os.environ.get("student_url")

professor_url ="%20https%3A//drive.google.com/uc%3Fexport%3Ddownload%26id%3D19GFc2yNb-voEjsczGtxcK_No4XGDS7Wp"
student_url ="%20https%3A//drive.google.com/uc%3Fexport%3Ddownload%26id%3D1Jp3Ppinl7HRPn4XwKcMj6rFMvcfzfp1p"

print("Professor URL:", professor_url)
print("Student URL:", student_url)


Professor URL: %20https%3A//drive.google.com/uc%3Fexport%3Ddownload%26id%3D19GFc2yNb-voEjsczGtxcK_No4XGDS7Wp
Student URL: %20https%3A//drive.google.com/uc%3Fexport%3Ddownload%26id%3D1Jp3Ppinl7HRPn4XwKcMj6rFMvcfzfp1p


In [ ]:
# Load audio files
prof_audio, sr1 = load_audio_from_url(professor_url)
stud_audio, sr2 = load_audio_from_url(student_url)

# Resample if needed
if sr1 != sr2:
    stud_audio = librosa.resample(stud_audio, orig_sr=sr2, target_sr=sr1)
    sr2 = sr1

🔗 Attempting to download: %20https%3A//drive.google.com/uc%3Fexport%3Ddownload%26id%3D19GFc2yNb-voEjsczGtxcK_No4XGDS7Wp


RuntimeError: ❌ Failed to load audio from %20https%3A//drive.google.com/uc%3Fexport%3Ddownload%26id%3D19GFc2yNb-voEjsczGtxcK_No4XGDS7Wp
Error: Invalid URL '%20https%3A//drive.google.com/uc%3Fexport%3Ddownload%26id%3D19GFc2yNb-voEjsczGtxcK_No4XGDS7Wp': No scheme supplied. Perhaps you meant https://%20https%3A//drive.google.com/uc%3Fexport%3Ddownload%26id%3D19GFc2yNb-voEjsczGtxcK_No4XGDS7Wp?

In [ ]:
# Extract pitch using yin (more robust and stable than pyin)
prof_pitch = librosa.yin(prof_audio, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'), sr=sr1)
stud_pitch = librosa.yin(stud_audio, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'), sr=sr2)


In [ ]:
# Plot pitch comparison
plt.figure(figsize=(14, 5))
plt.plot(prof_pitch, label='Professor Pitch', alpha=0.75)
plt.plot(stud_pitch, label='Student Pitch', alpha=0.75)
plt.legend()
plt.title("Pitch Comparison")
plt.xlabel("Frame")
plt.ylabel("Pitch (Hz)")
plt.grid(True)
plt.show()

# Trim pitch arrays to the same length
min_len = min(len(prof_pitch), len(stud_pitch))
prof_pitch_clean = np.nan_to_num(prof_pitch[:min_len])
stud_pitch_clean = np.nan_to_num(stud_pitch[:min_len])

# Basic pitch error analysis
pitch_diff = np.abs(prof_pitch - stud_pitch[:len(prof_pitch)])
avg_error = np.mean(pitch_diff)



In [ ]:
# Plot the pitch contours
plt.figure(figsize=(14, 5))
plt.plot(prof_pitch_clean, label='Professor Pitch', alpha=0.75)
plt.plot(stud_pitch_clean, label='Student Pitch', alpha=0.75)
plt.legend()
plt.title("Pitch Comparison")
plt.xlabel("Frame")
plt.ylabel("Pitch (Hz)")
plt.grid(True)
plt.show()

In [ ]:
print(f"\n🎵 Average pitch difference: {avg_error:.2f} Hz")
if avg_error < 20:
    print("🎯 Great job! Your pitch closely matches the reference.")
elif avg_error < 50:
    print("👍 You're in the ballpark, but there’s room for improvement.")
else:
    print("⚠️ Your pitch deviates significantly. Focus on tuning and accuracy.")


